# Setup

In [40]:
# Basics
import html
import json
import re
from pathlib import Path
import warnings

# Data Manipulation
import pandas as pd

# EDA
from unidecode import unidecode
from klib import convert_datatypes
from ydata_profiling import ProfileReport
import ppscore as pps
import dtreeviz

# Feature Engineering
from feature_engine.encoding import OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder as SkOrdinalEncoder 
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.encoding import CountFrequencyEncoder, OneHotEncoder
from feature_engine.pipeline import make_pipeline

# Machine Learning
from sklearn.tree import DecisionTreeRegressor

In [41]:
# Constants
PROJECT_PATH = Path.cwd().parent.resolve()
DATA_PATH = PROJECT_PATH.joinpath("data")
DATA_RAW_PATH = DATA_PATH.joinpath("raw")
DATA_PROCESSED_PATH = DATA_PATH.joinpath("processed")
RESOURCES_PATH = PROJECT_PATH.joinpath("resources")

In [42]:
# Data anonimization
store_name_encoder = OrdinalEncoder(
    encoding_method="arbitrary",
    variables=["marca_da_loja", "nome_da_loja"],
)

In [43]:
# Metadata
with open(DATA_RAW_PATH.joinpath("raw_data_dict.json")) as json_file:
    metadata = json.load(json_file)

dtype_mapping = {
    item["nome_do_campo_antes_limpeza"]: item["tipo"]
    for item in metadata
    if item["tipo"] != "datetime64[ns]"
}

date_columns = [
    item["nome_do_campo_antes_limpeza"]
    for item in metadata
    if item["tipo"] == "datetime64[ns]"
]

variables_descriptions = {
    item["nome_do_campo_apos_limpeza"]: item["descrição"]
    for item in metadata
}

column_names_mapping = {
    item["nome_do_campo_antes_limpeza"]: item["nome_do_campo_apos_limpeza"]
    for item in metadata
}

categorical_columns = [
    item["nome_do_campo_apos_limpeza"]
    for item in metadata
    if item["tipo"] == "category"
]

columns_to_keep = [
    item["nome_do_campo_apos_limpeza"]
    for item in metadata
    if item["disponivel_na_hora_do_pedido"] == True
]

# Ingestion

In [44]:
# Ingestion
temp_dfs = []

for file in list(DATA_RAW_PATH.glob("relatorio-logistica_*.xlsx")):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning, module=re.escape('openpyxl.styles.stylesheet'))
        df = (
            pd
            .read_excel(
                io=file,
                dtype=dtype_mapping,
                engine="openpyxl",
                na_values="",
                parse_dates=date_columns,
                date_format="%Y-%m-%d %H:%M:%S",
            )
            .pipe(lambda _df: _df.set_axis([html.unescape(col) for col in _df.columns], axis=1))
            .rename(columns=column_names_mapping)
            .assign(**{
                col: lambda _df, col=col: _df[col].apply(
                    lambda x: unidecode(html.unescape(str(x))) if pd.notna(x) else x
                )
                for col in categorical_columns
            })
        )
        temp_dfs.append(df)

df = (
    store_name_encoder.fit_transform(
        X=convert_datatypes(
            pd
            .concat(
                temp_dfs, 
                ignore_index=True
            )
            .loc[
                lambda _df: _df["status_final_do_pedido"].ne("CANCELADO"), 
                columns_to_keep
            ]
            .drop(
                columns=[
                    # Business decision
                    "tempo_prometido_de_entrega_min",
                    "tempo_de_atraso_em_relacao_ao_tempo_prometido_de_entrega_min",
                    "frete_cobrado_do_restaurante_apenas_sob_demanda",
                    # No value added
                    "id_da_loja", 
                    "id_curto_do_pedido",
                    "id_completo_do_pedido",
                    # Constant after filtering
                    "status_final_do_pedido",  
                ]
            )
        )
    )
    .assign(
        marca_da_loja=lambda _df: _df["marca_da_loja"].astype("category"),
        nome_da_loja=lambda _df: _df["nome_da_loja"].astype("category"),
    )   
)

# EDA

## Preprocessing

In [45]:
# Datetime features creation
df = (
    df
    .assign(
        dia_da_semana=lambda _df: _df["data_e_hora_do_pedido"].dt.dayofweek,
        dia_do_mes=lambda _df: _df["data_e_hora_do_pedido"].dt.day,
        hora=lambda _df: _df["data_e_hora_do_pedido"].dt.hour,
        minuto=lambda _df: _df["data_e_hora_do_pedido"].dt.minute,
        minutos_desde_meia_noite=lambda _df: (
            _df["data_e_hora_do_pedido"].dt.hour.mul(60).add(_df["data_e_hora_do_pedido"].dt.minute)
        ),
    )
)

## Overall Report

In [46]:
ProfileReport(
    df=df,
    title="deliveries_data",
    variables={"descriptions": variables_descriptions},
    vars={"cat": {"length": False}},
    interactions={"continuous": True, "targets": ["tempo_da_entrega_realizada_min"]}
).to_file(output_file=RESOURCES_PATH.joinpath("reports", "deliveries_data_profile_report.html"))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:00<00:00, 748.27it/s]
/Users/ewerthon/Documents/Materiais/data_science/mestrado/reconhecimento_de_padroes/delivery-time-prediction/.venv/lib/python3.12/site-packages/ydata_profiling/model/pandas/discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[8 8 8 ... 4 4 4]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
/Users/ewerthon/Documents/Materiais/data_science/mestrado/reconhecimento_de_padroes/delivery-time-prediction/.venv/lib/python3.12/site-packages/ydata_profiling/model/pandas/discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 2 2 ... 1 1 1]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
/Users/ewerthon/Documents/Materiais/data_science/mestrado/reconhecimento_de_padroes/delivery-time-predictio

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Predictive Power Score

In [47]:
# Encoders for Preprocessing
shift_encoder = SklearnTransformerWrapper(transformer=SkOrdinalEncoder(categories=[["MANHA", "ALMOCO", "TARDE", "JANTAR", "CEIA", "MADRUGADA"]]), variables=["turno"])
priority_encoder = SklearnTransformerWrapper(transformer=SkOrdinalEncoder(categories=[["PADRAO", "RAPIDA"]]), variables=["prioridade_do_pedido"])
brand_and_store_encoder = CountFrequencyEncoder(encoding_method="count", variables=["marca_da_loja", "nome_da_loja"])
logistic_service_encoder = OneHotEncoder(drop_last=True, variables=["servico_logistico"])

pipeline = make_pipeline(
    shift_encoder,
    priority_encoder,
    brand_and_store_encoder,
    logistic_service_encoder,
)

df_encoded = pipeline.fit_transform(df.drop(columns=["data_e_hora_do_pedido"]))

df_pps_predictors = (
    pps.predictors(
        df=df_encoded,
        y="tempo_da_entrega_realizada_min",
    )
    .loc[lambda df_: df_["ppscore"] > 0, :]
    .drop(columns=["model", "y", "case", "is_valid_score", "metric"])
    .sort_values(by="ppscore", ascending=False)
)

df_pps_predictors.to_parquet(path=DATA_PROCESSED_PATH.joinpath("pps_predictors.parquet"), engine="pyarrow", index=False)

In [48]:
# Visualization
display(df_pps_predictors)

,x,ppscore,baseline_score,model_score
0,taxa_de_entrega_paga_pelo_cliente_reais,0.041330,12.422264,11.908855
1,servico_logistico_FULL_SERVICE,0.003133,12.422264,12.383340


## Tree Visualization

In [49]:
tree_regressor = DecisionTreeRegressor(max_depth=4, random_state=42)

tree_regressor.fit(
    X=df_encoded.drop(columns=["tempo_da_entrega_realizada_min"]), 
    y=df_encoded[["tempo_da_entrega_realizada_min"]]
)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    tree_viz_model = dtreeviz.model(
        model=tree_regressor,
        X_train=df_encoded.drop(columns=["tempo_da_entrega_realizada_min"]),
        y_train=df_encoded["tempo_da_entrega_realizada_min"],
        target_name="tempo_da_entrega_realizada_min",
        feature_names=df_encoded.drop(columns=["tempo_da_entrega_realizada_min"]).columns.tolist(),
    )

    tree_viz_model.view(orientation="LR").save(filename=str(RESOURCES_PATH.joinpath("visualizations", "tree_regressor.svg")))

# Feature Engineering

In [50]:
def count_orders_60min_vectorized(group):
    counts = []
    datetimes = group["data_e_hora_do_pedido"].values
    
    for _, current_time in enumerate(datetimes):
        time_60min_ago = current_time - pd.Timedelta(minutes=60)
        count = ((datetimes < current_time) & (datetimes >= time_60min_ago)).sum()
        counts.append(count)
    
    group = group.copy()
    group["pedidos_ultimos_60min"] = counts
    return group

df = (
    df
    .sort_values(["nome_da_loja", "data_e_hora_do_pedido"])
    .groupby("nome_da_loja", group_keys=False, observed=True)
    [df.columns]
    .apply(count_orders_60min_vectorized)
    .sort_index()
)

In [51]:
df

,marca_da_loja,nome_da_loja,data_e_hora_do_pedido,turno,servico_logistico,prioridade_do_pedido,tempo_da_entrega_realizada_min,distancia_percorrida_ate_o_cliente_km,taxa_de_entrega_paga_pelo_cliente_reais,valor_total_dos_itens_do_pedido_reais,dia_da_semana,dia_do_mes,hora,minuto,minutos_desde_meia_noite,pedidos_ultimos_60min
0,0,0,2025-08-09 11:56:12,ALMOCO,SOB_DEMANDA_ON,PADRAO,31.650000,1.93,0.00,43.900002,5,9,11,56,716,5
1,0,0,2025-08-09 11:23:36,ALMOCO,SOB_DEMANDA_ON,PADRAO,30.120001,0.67,0.00,90.800003,5,9,11,23,683,2
2,0,1,2025-08-09 12:07:20,ALMOCO,SOB_DEMANDA_ON,PADRAO,16.030001,0.66,0.00,59.900002,5,9,12,7,727,3
3,0,1,2025-08-09 14:46:55,TARDE,SOB_DEMANDA_ON,PADRAO,15.470000,1.88,6.99,70.900002,5,9,14,46,886,1
4,0,0,2025-08-09 20:54:50,JANTAR,SOB_DEMANDA_ON,PADRAO,62.900002,4.31,3.00,255.899994,5,9,20,54,1254,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19546,0,1,2025-11-06 19:51:42,JANTAR,ENTREGA_MAIS_FLEX,PADRAO,51.599998,11.65,20.99,135.800003,3,6,19,51,1191,40
19547,0,1,2025-11-06 22:27:52,CEIA,ENTREGA_MAIS_FLEX,PADRAO,31.530001,8.49,19.99,60.400002,3,6,22,27,1347,13
19548,0,1,2025-11-06 22:11:40,CEIA,ENTREGA_MAIS_FLEX,PADRAO,26.480000,6.00,12.99,63.400002,3,6,22,11,1331,17
19549,0,0,2025-11-06 22:44:10,CEIA,ENTREGA_MAIS_FLEX,PADRAO,40.549999,10.02,22.99,72.800003,3,6,22,44,1364,3


# Exporting

In [52]:
df.to_parquet(
    path=DATA_PROCESSED_PATH.joinpath("deliveries.parquet"),
    engine="pyarrow",
    index=False
)